# hit-song-classifier
## Random Forest Classification

The first thing we'll do is define our imports and declare any constants.

In [1]:
# Define our imports
import numpy as np
import sklearn

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Define our constants
TARGET_DATA = "../../data/join_datasets1.csv"
RANDOM_SEED = 0

Next, we shall build our set of inputs X. We don't want to include the features `[song, artist, year]` since they are keys. We also should leave feature `class` out since those are our classification marker.

In [ ]:
# Import our data
